## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 18

### sub-modelを使用するかどうか

In [3]:
use_sub_model = True

In [4]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
SUBMIT = 'submit/course_{}.csv'.format(model_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/all_merge_{}_sub.f'.format(model_No)
    SUBMIT = 'submit/course_{}_sub.csv'.format(model_No)
else:
    OUT_SUBMODEL = 'intermediate/course_predict_{}.f'.format(model_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 238)

In [6]:
if use_sub_model:
    all_pitch.drop(columns=[
        'predict_0',
        'predict_1',
        'predict_2',
        'predict_3',
        'predict_4',
        'predict_5',
        'predict_6',
        'predict_7',
        'predict_8',
        'predict_9',
        'predict_10',
        'predict_11',
        'predict_12'
    ], inplace=True)

In [7]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 225)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course00,bc_course01,bc_course02,bc_course03,bc_course04,bc_course05,bc_course06,bc_course07,bc_course08,bc_course09,bc_course10,bc_course11,bc_course12,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,...,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,predict_straight,predict_curve,predict_slider,predict_shoot,predict_fork,predict_changeup,predict_sinker,predict_cutball
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.824838,0.017786,0.034873,0.121010,0.000158,0.001322,0.000004,0.000009
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.652693,0.092909,0.101409,0.116226,0.008166,0.028534,0.000017,0.000047
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0.076007,0.284249,0.102015,0.095055,0.442674,0.118315,0.488095,0.076190,0.078571,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.551756,0.182159,0.094477,0.040574,0.081375,0.049591,0.000032,0.000035
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0.151308,0.196937,0.206965,0.157643,0.287147,0.105057,0.153718,0.164756,0.246104,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1,0.606935,0.093023,0.182139,0.117593,0.000087,0.000186,0.000004,0.000032
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.0495

### train
- 行数: 257117

In [8]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 225)

### test
- 行数: 521650

In [9]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 225)

In [10]:
del all_pitch
gc.collect()

60

In [11]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [12]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [13]:
lgb_train = lgb.Dataset(train_d, train['course'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [14]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 8.769293390201968, 
        'lambda_l2': 3.913949617576324e-05, 
        'num_leaves': 6, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.8391111798378441, 
        'bagging_freq': 4, 
        'min_child_samples': 50
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.31677 + 0.000891962
[200]	cv_agg's multi_logloss: 2.31122 + 0.00089479
[300]	cv_agg's multi_logloss: 2.30942 + 0.00088671
[400]	cv_agg's multi_logloss: 2.30857 + 0.000928233
[500]	cv_agg's multi_logloss: 2.30817 + 0.00092368
[600]	cv_agg's multi_logloss: 2.30817 + 0.000921354
Best num_boost_round: 548
Best CV score: 2.308116414786115
CPU times: user 1h 6min 58s, sys: 12.3 s, total: 1h 7min 10s
Wall time: 8min 25s


In [15]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [16]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 19min 43s, sys: 3.29 s, total: 19min 46s
Wall time: 2min 28s


### Feature Importance

In [17]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
135,course_11,341
119,fork,349
152,weight_bat,352
168,bat_game_cnt,353
151,height_bat,354
61,elapsed_batter,359
166,bmi_bat,371
134,course_10,373
118,shoot,374
193,bmi_dif_p-b,376


In [18]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 3min 50s, sys: 912 ms, total: 3min 51s
Wall time: 30.3 s


## 結果出力

In [19]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.064043,0.139287,0.061913,0.036143,0.058246,0.027586,0.062082,0.052522,0.034929,0.182558,0.095179,0.088289,0.097221
1,1,0.055680,0.143546,0.063389,0.031989,0.052166,0.025993,0.045915,0.057937,0.038777,0.162578,0.099537,0.109298,0.113197
2,2,0.027374,0.097189,0.056489,0.021184,0.042098,0.022345,0.034388,0.047834,0.030680,0.141402,0.089224,0.234754,0.155040
3,3,0.042733,0.104082,0.057214,0.024771,0.039796,0.022362,0.042261,0.054811,0.041968,0.160580,0.118838,0.144216,0.146369
4,4,0.047774,0.121567,0.070034,0.036016,0.053097,0.035536,0.046274,0.068606,0.054025,0.131710,0.067995,0.112278,0.155089


In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### 球種予測で使用する

In [21]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [22]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_0',
        1: 'predict_1',
        2: 'predict_2',
        3: 'predict_3',
        4: 'predict_4',
        5: 'predict_5',
        6: 'predict_6',
        7: 'predict_7',
        8: 'predict_8',
        9: 'predict_9',
        10: 'predict_10',
        11: 'predict_11',
        12: 'predict_12'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

In [23]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

### SIGNATE

In [24]:
'signate submit --competition-id=276 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ./submit/course_18_sub.csv --note feat=225_cv=2.308116414786115'

### GCS

In [25]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/course_18_sub.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 18_sub(225) ...サブモデルで球種の予測を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 548
- Best CV score: 2.308116414786115
- Wall time: 8min 25s
- 評価結果  : 2.3638683

#### 18(217) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 373
- Best CV score: 2.3204478805445055
- Wall time: 6min 9s
- 評価結果  : 2.3604751

#### 17(192) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 415
- Best CV score: 2.3200530570975535
- Wall time: 15min 10s
- 評価結果  : 2.3607863

#### 16(181)  ...選手IDごとの2017年のコース実績
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 355
- Best CV score: 2.3203553166835866
- Wall time: 5min 8s
- 評価結果  : 2.3608998

#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543 ★

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648